# KNN-Airbnb

利用Airbnb 資料來學習knn

In [9]:
import pandas as pd
import numpy as np
dc_listings = pd.read_csv('Downloads/listings.csv')
dc_listings.head(5)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,7087327,https://www.airbnb.com/rooms/7087327,20151002231825,2015-10-03,Historic DC Condo-Walk to Capitol!,Professional pictures coming soon! Welcome to ...,NaN,Professional pictures coming soon! Welcome to ...,none,NaN,...,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,18,NaN
1,975833,https://www.airbnb.com/rooms/975833,20151002231825,2015-10-03,Spacious Capitol Hill Townhouse,NaN,Beautifully renovated Capitol Hill townhouse. ...,Beautifully renovated Capitol Hill townhouse. ...,none,NaN,...,9.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,strict,f,f,1,2.11
2,8249488,https://www.airbnb.com/rooms/8249488,20151002231825,2015-10-03,Spacious/private room for single,This is an ideal room for a single traveler th...,NaN,This is an ideal room for a single traveler th...,none,NaN,...,NaN,f,NaN,NaN,f,flexible,f,f,1,1.00
3,8409022,https://www.airbnb.com/rooms/8409022,20151002231825,2015-10-03,A wonderful bedroom with library,Prime location right on the Potomac River in W...,NaN,Prime location right on the Potomac River in W...,none,NaN,...,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,1,NaN
4,8411173,https://www.airbnb.com/rooms/8411173,20151002231825,2015-10-03,Downtown Silver Spring,"Hi travellers! I live in this peaceful spot, b...",This is a 750 sq ft 1 bedroom 1 bathroom. Whi...,"Hi travellers! I live in this peaceful spot, b...",none,Silver Spring is booming. You can walk to a n...,...,NaN,f,NaN,NaN,f,flexible,f,f,1,NaN


主要用accommodats來預測price


In [10]:
#首先先將price 欄位轉換為float

stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')
dc_listings['price'].head(5)

0    160.0
1    350.0
2     50.0
3     95.0
4     50.0
Name: price, dtype: float64

In [11]:
#將index隨機排序
#If we sort by the distance column and then just select the first 5 living spaces, we would be biasing the result to 
dc_listings = dc_listings.loc[np.random.permutation(len(dc_listings))]
dc_listings.head(10)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
93,7826994,https://www.airbnb.com/rooms/7826994,20151002231825,2015-10-03,Great price for DC room,The room is a large suite. There is a queen s...,"This space is outside the city center, but you...",The room is a large suite. There is a queen s...,none,"This neighborhood is quiet, close to River Ter...",...,9.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,2,2.73
1915,7918040,https://www.airbnb.com/rooms/7918040,20151002231825,2015-10-03,Positive Energy Home & Space!,This space is full of positive energy and affi...,NaN,This space is full of positive energy and affi...,none,NaN,...,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,1,NaN
1281,3592252,https://www.airbnb.com/rooms/3592252,20151002231825,2015-10-03,Charming DUPONT CIRCLE 1 Bedroom,We have a delightful and charming 1 bedroom lo...,We have a delightful and charming 1 bedroom lo...,We have a delightful and charming 1 bedroom lo...,none,NaN,...,9.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,moderate,f,f,1,1.71
650,925327,https://www.airbnb.com/rooms/925327,20151002231825,2015-10-03,"Modern, Sunny Studio Apt Downtown",NaN,This recently constructed studio apartment com...,This recently constructed studio apartment com...,none,NaN,...,9.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,moderate,f,f,1,0.26
3066,1868037,https://www.airbnb.com/rooms/1868037,20151002231825,2015-10-03,Cherry Blossom Getaway! Entire home,Stay with me in my spacious townhome. Centrall...,My place is super spacious. There are two roo...,Stay with me in my spacious townhome. Centrall...,none,"Comfortable neighborhood, close to downtown.",...,10.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,moderate,f,f,1,0.19
585,7996960,https://www.airbnb.com/rooms/7996960,20151002231825,2015-10-03,Renovated STUDIO (Logan Circle)!,Fully renovated spacious studio. Conveniently ...,NaN,Fully renovated spacious studio. Conveniently ...,none,NaN,...,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,moderate,f,f,1,NaN
1226,211986,https://www.airbnb.com/rooms/211986,20151002231825,2015-10-03,Modern B&B room-3 min walk to metro,See more pictures and contact information by s...,"The Ella Jo room has a queen-size bed, and twi...",See more pictures and contact information by s...,none,The Ivy Mansion is two or three short blocks f...,...,9.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,strict,f,f,4,0.36
142,3477770,https://www.airbnb.com/rooms/3477770,20151002231825,2015-10-03,Spacious Home in Downtown DC,"Located steps from Chinatown, the National Mal...","At over 2k square feet, this space is ideal fo...","Located steps from Chinatown, the National Mal...",none,The unit is perfectly situated in the heart of...,...,10.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",t,strict,f,f,2,2.34
1128,5314716,https://www.airbnb.com/rooms/5314716,20151002231825,2015-10-03,"Large, Sunny Apt in Dupont Circle",NaN,"Huge, sunny and quiet, this beautiful 1 bedroo...","Huge, sunny and quiet, this beautiful 1 bedroo...",none,"Dupont Circle is known for its art galleries, ...",...,10.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",t,moderate,f,f,1,0.55
3434,6001758,https://www.airbnb.com/rooms/6001758,20151002231825,2015-10-03,"Capital Hill, by Metro, Sleeps 10!",Quiet 3 bdrm house on Capital Hill. Can sleep ...,***PLEASE READ MY ENTIRE LISTING. You do not n...,Quiet 3 bdrm house on Capital Hill. Can sleep ...,none,GREAT LOCATION! My neighborhood is in a great ...,...,9.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",t,strict,f,f,1,3.62


In [12]:
'''
透過寫一個function來寫出預測價格的功能
1.此function須有一個parameter-new_listing 代表房間數量
2.將原始的dataset做一個copy(不影響原是資料)，並將accommodates 欄位與 new_listing 相減得到距離 distance 欄位
3.再將df依照距離由短到長排序，找出前五名做平均->得到的就是預測值
'''
def predict_price(new_listing):
    temp_df = dc_listings.copy()
    ## Complete the function.
    temp_df['distance']=temp_df['accommodates'].apply(lambda x:abs(x-new_listing))
    temp_df=temp_df.sort_values(by=['distance'])
    predict=temp_df['price'].iloc[0:5].mean()
    return predict


In [15]:
acc_one = predict_price(1)
acc_two = predict_price(2)
acc_four = predict_price(4)
print('acc_one',acc_one)
print('acc_two',acc_two)
print('acc_four',acc_four)

acc_one 102.0
acc_two 86.8
acc_four 145.6


# How to evaluate the predict price
在這邊主要會介紹MAE,MSE,RMSE這幾中統計判斷誤差


A simple way to test the quality of your model is to:

split the dataset into 2 partitions:

the training set: contains the majority of the rows (75%)
the test set: contains the remaining minority of the rows (25%)

In [19]:

train_df = dc_listings.iloc[0:2792]
test_df = dc_listings.iloc[2792:]

def predict_price(new_listing):
    temp_df=train_df.copy()
    temp_df['distance']=temp_df['accommodates'].apply(lambda x:np.abs(x-new_listing))
    temp_df=temp_df.sort_values('distance')
    nearest_neighbor_prices=temp_df.iloc[0:5]['price']
    predict_price=nearest_neighbor_prices.mean()
    return predict_price

test_df['predicted_price']=test_df['accommodates'].apply(predict_price)
test_df['predicted_price'].head()

C:\Users\janice\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


2334    145.6
485     134.0
3645    134.0
338     123.2
2519    123.2
Name: predicted_price, dtype: float64

In [21]:
#mae: sum(predict-price).mean
mae=(np.absolute(test_df['predicted_price']-test_df['price'])).mean()
mae

75.90547798066596

In [22]:
#mse
mse=((test_df['predicted_price']-test_df['price'])**2).mean()

test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
mse = test_df['squared_error'].mean()
print(mse)

19984.28597207303


C:\Users\janice\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [23]:
#使用另一個因素bathrooms
def predict_price(new_listing):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['bathrooms'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbors_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbors_prices.mean()
    return(predicted_price)

test_df['predicted_price']=test_df['bathrooms'].apply(predict_price)
test_df['squared_error']=(test_df['predicted_price']-test_df['price'])**2
mse=test_df['squared_error'].mean()
print(mse)

17022.859849624027


C:\Users\janice\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\janice\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [25]:
rmse=mse**(0.5)
rmse

130.4716821751909